# ***Loading Module***

In [1]:
import pandas as pd
import mysql.connector
from mysql.connector import connect, Error
from getpass import getpass
import warnings
from urllib.parse import quote_plus
warnings.filterwarnings("ignore", category=UserWarning, module="pandas") # Suppress warnings from pandas
from sqlalchemy import create_engine
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px


# *Connecting To MySQL*

In [2]:
# Get user input for username and password
username = input("Enter username: ")
password = getpass("Enter password: ")

# Create a connection to the MySQL server
try:
    with connect(
        host="localhost",
        user=username,
        password=password,
    ) as connection:
        print("Connection to MySQL server established.")

except Exception as e:
    print(f"Error connecting to MySQL server: {e}")

Connection to MySQL server established.


In [3]:
connection.reconnect()

# *Creating Database*

In [4]:
# Create database if it doesn't exist
create_database_query = "CREATE DATABASE IF NOT EXISTS analyze_personal_expenses"
with connection.cursor() as cursor:
    cursor.execute(create_database_query)
    connection.commit()
    print("Database Created Successfully")

Database Created Successfully


# *Connecting To An Existing Database*

In [5]:
# Connecting to an existing database.


database = mysql.connector.connect(
    host ="localhost",
    username = input("Enter username"),
    password = getpass("Enter password"),
    database = "analyze_personal_expenses"
)

my_cursor = database.cursor()

# *Reading CSV File.*

In [7]:
query_data = pd.read_csv("combined_data.csv")
query_data.head()

,Date,Category,Payment Mode,Description,Amount Paid,Cashback
0,2023-01-01,Clothing,Cash,Athletic Wear,2451.52,93.64
1,2023-01-01,Health & Fitness,Online,Personal Training Sessions,3199.94,46.80
2,2023-01-01,Subscriptions,Cash,Cloud Storage Services,459.39,34.12
3,2023-01-01,Clothing,Online,Formal Wear,2065.88,76.83
4,2023-01-02,Clothing,Cash,Fashion Accessories,350.35,33.80


# *Creating Table*

In [ ]:
# SQL query to create the expenses table
create_expenses_table_query = """
CREATE TABLE IF NOT EXISTS personal_expenses (
    id INT AUTO_INCREMENT PRIMARY KEY,
    Date DATETIME NOT NULL,
    Category VARCHAR(250) NOT NULL,
    `Payment Mode` VARCHAR(100) NOT NULL,
    Description VARCHAR(250) NOT NULL,
    `Amount Paid` DECIMAL(10, 2) NOT NULL,
    Cashback DECIMAL(10, 2) NOT NULL
);
"""

# Execute the create table query
my_cursor.execute(create_expenses_table_query)
connection.commit()

# *Inserting Data Into Table*

In [10]:
insert_query = """
INSERT INTO personal_expenses (Date, Category, `Payment Mode`, Description, `Amount Paid`, Cashback)
VALUES (%s, %s, %s, %s, %s, %s)
"""

for _, row in query_data.iterrows():
    try:
        my_cursor.execute(insert_query, tuple(row))
        connection.commit()
    except Exception as e:
        print(f"Error inserting row {row}: {e}")

# *Running Queries*

In [11]:
query1 = ("SELECT COUNT(*) FROM PERSONAL_EXPENSES")

my_cursor.execute(query1)
result = my_cursor.fetchall()
for x in result:
    print(x)

(2400,)


In [12]:
query2 = ("SELECT SUM(`Amount Paid`) FROM PERSONAL_EXPENSES")

my_cursor.execute(query2)
result = my_cursor.fetchall()
for x in result:
    print(x)

(Decimal('5891350.56'),)


In [13]:
# Spending By Category
query3 =("SELECT Category, SUM(`Amount Paid`) AS total_spent FROM PERSONAL_EXPENSES GROUP BY category")


my_cursor.execute(query3)
result = my_cursor.fetchall()
for x in result:
    print(x)
    
grouped = pd.DataFrame(result,columns=["Category","Amount Paid"])
# Bar Chart

fig = px.bar(grouped, x="Category",y="Amount Paid", color="Category", title="Total Amount Paid by Category",text="Amount Paid")

fig.show()

('Clothing', Decimal('412680.32'))
('Health & Fitness', Decimal('330817.90'))
('Subscriptions', Decimal('227428.30'))
('Groceries', Decimal('230642.42'))
('Dining Out', Decimal('420679.34'))
('Housing', Decimal('2113226.62'))
('Utilities', Decimal('231095.58'))
('Transportation', Decimal('429892.66'))
('Entertainment', Decimal('380238.56'))
('Travel', Decimal('1114648.86'))


In [14]:
query4 = ("SELECT `Payment Mode`, COUNT(*) FROM PERSONAL_EXPENSES GROUP BY `Payment Mode`")

my_cursor.execute(query4)

result = my_cursor.fetchall()

for x in result:
    
    print(x)
    
df_piechart = pd.DataFrame(result, columns=['Payment Mode', 'Count'])
    
fig = px.pie(df_piechart, values='Count', names='Payment Mode', 
             title='Distribution of Payment Modes',
             color='Payment Mode',
             color_discrete_sequence=px.colors.sequential.Rainbow
)
fig.show()

('Cash', 1132)
('Online', 1268)


In [15]:
query5 = ("SELECT Description, SUM(`Amount Paid`) FROM PERSONAL_EXPENSES GROUP BY Description ORDER BY SUM(`Amount Paid`) DESC LIMIT 10")


my_cursor.execute(query5)

top_expenses_result = my_cursor.fetchall()

for x in top_expenses_result:
    
    print(x)
    
top_expenses_df = pd.DataFrame(top_expenses_result, columns=["Description", "Total Amount Paid"])

fig  = px.bar(top_expenses_df,x = "Description", y = "Total Amount Paid",
            title="Top 10 Description By Total Amount Paid",
            color= "Total Amount Paid",
            text= "Total Amount Paid",
            color_continuous_scale=px.colors.sequential.Inferno)
    
fig.show()


('Mortgage Payment', Decimal('733978.94'))
('Airfare', Decimal('553544.78'))
('Rent Payment', Decimal('497809.30'))
('Appliance Purchases', Decimal('381297.72'))
('Furnishings', Decimal('276569.18'))
('Hotel Accommodation', Decimal('248415.68'))
('Dinner At A Restaurant', Decimal('194050.90'))
('Fuel Costs', Decimal('131168.46'))
('Video Games', Decimal('105345.72'))
('Tour Activities', Decimal('88625.32'))


In [16]:
# Transactions with amount > 40000

query6 = ("SELECT * FROM PERSONAL_EXPENSES WHERE `Amount Paid` > 40000")

my_cursor.execute(query6)

transaction_above_40000 = my_cursor.fetchall()

for x in transaction_above_40000:
    
    print(x)
    
transaction_above_40000_df =pd.DataFrame(transaction_above_40000)
    
column_names = [i[0] for i in my_cursor.description]
transaction_above_40000_df.columns = column_names 

fig = px.bar(transaction_above_40000_df, x="Category", y="Amount Paid",
             title="Transactions Above 40,000 by Category",
             color="Amount Paid",
             text="Amount Paid",
             color_discrete_sequence= px.colors.sequential.Plasma_r)

fig.show()


(487, datetime.date(2023, 5, 25), 'Travel', 'Online', 'Airfare', Decimal('42455.96'), Decimal('3919.34'))
(598, datetime.date(2023, 6, 30), 'Travel', 'Online', 'Airfare', Decimal('43043.98'), Decimal('599.01'))
(626, datetime.date(2023, 7, 8), 'Travel', 'Cash', 'Airfare', Decimal('40160.03'), Decimal('2001.71'))
(731, datetime.date(2023, 8, 9), 'Travel', 'Online', 'Airfare', Decimal('40630.31'), Decimal('3097.97'))
(893, datetime.date(2023, 9, 27), 'Travel', 'Online', 'Airfare', Decimal('45263.34'), Decimal('1402.38'))
(1687, datetime.date(2023, 5, 25), 'Travel', 'Online', 'Airfare', Decimal('42455.96'), Decimal('3919.34'))
(1798, datetime.date(2023, 6, 30), 'Travel', 'Online', 'Airfare', Decimal('43043.98'), Decimal('599.01'))
(1826, datetime.date(2023, 7, 8), 'Travel', 'Cash', 'Airfare', Decimal('40160.03'), Decimal('2001.71'))
(1931, datetime.date(2023, 8, 9), 'Travel', 'Online', 'Airfare', Decimal('40630.31'), Decimal('3097.97'))
(2093, datetime.date(2023, 9, 27), 'Travel', 'Online

In [17]:
# Daily spending trend

query7 = "SELECT date, SUM(`Amount Paid`) FROM PERSONAL_EXPENSES GROUP BY date ORDER BY date"

my_cursor.execute(query7)

daily_spending_trend = my_cursor.fetchall()

for x in daily_spending_trend:
    
    print(x)
    
    
daily_spending_trend_df = pd.DataFrame(daily_spending_trend, columns=["Date", "Amount Paid"])

fig = px.line(daily_spending_trend_df,
              x="Date",
              y="Amount Paid",
              title="Amount Paid Over Time",
              markers=True)

fig.update_traces(
    line=dict(color="#003366", width=3),  # Dark Blue
    marker=dict(color="#FFD700", size=5)   # Gold
)

fig.update_layout(xaxis_tickangle=-90)

fig.show()

(datetime.date(2023, 1, 1), Decimal('16353.46'))
(datetime.date(2023, 1, 2), Decimal('700.70'))
(datetime.date(2023, 1, 3), Decimal('4411.02'))
(datetime.date(2023, 1, 4), Decimal('1530.38'))
(datetime.date(2023, 1, 5), Decimal('3894.10'))
(datetime.date(2023, 1, 6), Decimal('45354.06'))
(datetime.date(2023, 1, 7), Decimal('5061.80'))
(datetime.date(2023, 1, 8), Decimal('72604.60'))
(datetime.date(2023, 1, 9), Decimal('10370.42'))
(datetime.date(2023, 1, 10), Decimal('20682.64'))
(datetime.date(2023, 1, 11), Decimal('675.18'))
(datetime.date(2023, 1, 12), Decimal('3876.92'))
(datetime.date(2023, 1, 13), Decimal('31619.18'))
(datetime.date(2023, 1, 14), Decimal('4442.46'))
(datetime.date(2023, 1, 15), Decimal('13060.86'))
(datetime.date(2023, 1, 16), Decimal('23122.26'))
(datetime.date(2023, 1, 17), Decimal('10612.78'))
(datetime.date(2023, 1, 18), Decimal('18114.90'))
(datetime.date(2023, 1, 19), Decimal('7056.10'))
(datetime.date(2023, 1, 20), Decimal('9997.66'))
(datetime.date(2023, 

In [18]:
# Average spending per category.

query8 = ("SELECT Category, AVG(`Amount Paid`) FROM PERSONAL_EXPENSES GROUP BY Category")

my_cursor.execute(query8)

avg_spending_result = my_cursor.fetchall()

for x in avg_spending_result:
    
    print(x)
    
    
average_spending_df = pd.DataFrame(avg_spending_result, columns=["Category", "Average Spending"])

custom_colors = ['#78290f','#f7ede2','#bcbd8b','#b5e2fa','#eddea4',"#ae2012","#0d1321","#1b4965","#053c5e","#001f54"]


fig = px.bar(average_spending_df,
             x="Category",
             y="Average Spending",
             title="Average Spending Per Category",
             color="Average Spending",
             text="Average Spending",
             color_discrete_sequence=custom_colors,
             )

fig.show()



('Clothing', Decimal('1810.001404'))
('Health & Fitness', Decimal('1797.923370'))
('Subscriptions', Decimal('1093.405288'))
('Groceries', Decimal('1213.907474'))
('Dining Out', Decimal('808.998731'))
('Housing', Decimal('10358.954020'))
('Utilities', Decimal('1111.036442'))
('Transportation', Decimal('2047.107905'))
('Entertainment', Decimal('1558.354754'))
('Travel', Decimal('5463.965000'))


In [19]:
# Categories with more than 50 transactions

query9= ("SELECT Category, COUNT(*) FROM PERSONAL_EXPENSES GROUP BY Category HAVING COUNT(*) > 50")

my_cursor.execute(query9)

categories_above_50_transactions = my_cursor.fetchall()

for x in categories_above_50_transactions:
    
    print(x)
    
categories_df = pd.DataFrame(categories_above_50_transactions, columns=["Category", "Transaction Count"])

# Define a custom color sequence
custom_colors = ['#582f0e', '#7f4f24', '#936639', '#a68a64', '#d8572a','#373d20','#717744','#bcbd8b','#766153','#bcb8b1']

fig = px.bar(
    categories_df,
    x="Category",
    y="Transaction Count",
    title="Categories with More Than 50 Transactions",
    color="Category",
    text="Transaction Count",
    color_discrete_sequence=custom_colors
    )


fig.show()

('Clothing', 228)
('Health & Fitness', 184)
('Subscriptions', 208)
('Groceries', 190)
('Dining Out', 520)
('Housing', 204)
('Utilities', 208)
('Transportation', 210)
('Entertainment', 244)
('Travel', 204)


In [20]:
# Max amount spent in each category

query10 = ("SELECT Category, MAX(`Amount Paid`) FROM PERSONAL_EXPENSES GROUP BY Category")

my_cursor.execute(query10)

max_amount_spent = my_cursor.fetchall()

for x in max_amount_spent:
    
    print(x)
    
max_amount_df = pd.DataFrame(max_amount_spent, columns=["Category", "Max Amount Spent"])

category_color = ['#0466c8', '#0353a4', '#023e7d', '#002855', '#001845',
                '#001233', '#33415c', '#5c677d', '#7d8597', '#979dac']

fig = px.bar(
    max_amount_df,
    x="Category",
    y="Max Amount Spent",
    title="Maximum Amount Spent in Each Category",
    color="Category",  # Use Category for coloring
    color_discrete_sequence=category_color,
    text="Max Amount Spent"
    )

fig.show()

('Clothing', Decimal('8638.84'))
('Health & Fitness', Decimal('4968.95'))
('Subscriptions', Decimal('4634.93'))
('Groceries', Decimal('4022.83'))
('Dining Out', Decimal('4625.93'))
('Housing', Decimal('38218.03'))
('Utilities', Decimal('4579.84'))
('Transportation', Decimal('9966.81'))
('Entertainment', Decimal('4896.51'))
('Travel', Decimal('45263.34'))


In [21]:
# Most frequent descriptions

query11 = ("SELECT Description, COUNT(*) FROM PERSONAL_EXPENSES GROUP BY Description ORDER BY COUNT(*) DESC LIMIT 30")

my_cursor.execute(query11)

most_frequent_description = my_cursor.fetchall()

for frequency in most_frequent_description:

    print(frequency)

most_frequent_description_df = pd.DataFrame(most_frequent_description, columns=["Description","frequency"])

#description_color = ['#001219', '#005f73', '#0a9396', '#94d2bd', '#e9d8a6', 
                #'#ee9b00', '#ca6702', '#bb3e03', '#ae2012', '#9b2226']

fig= px.bar(most_frequent_description_df,
             x = "Description",
             y="frequency",
             title="Most Frequent Description In Personal Expenses",
             color="frequency",
             text="frequency",
             color_continuous_scale=px.colors.sequential.Magma)

fig.show()

('Coffee Shop Visit', 238)
('Dinner At A Restaurant', 100)
('Art Supplies', 40)
('Hobby Supplies', 38)
('Car Rentals', 34)
('Car Insurance', 34)
('Video Games', 34)
('Mortgage Payment', 32)
('Tour Activities', 32)
('Underwear And Socks', 30)
('Digital Magazine Subscriptions', 30)
('Brunch With Friends', 30)
('Beverages At Bars', 28)
('Sports Events Tickets', 28)
('Fashion Accessories', 28)
('Parking Fees', 28)
('Frozen Foods', 28)
('Rent Payment', 28)
('Maintenance Fees', 28)
('Vehicle Registration Fees', 28)
('Cleaning Supplies', 28)
('Online Learning Platforms', 28)
('Gym Membership', 28)
('Public Transit Pass', 26)
('Seasonal Clothing', 26)
('Baking Ingredients', 26)
('Bicycle Repairs', 26)
('Appliance Purchases', 26)
('Internet Service', 26)
('Tailoring Services', 26)


In [22]:
query12 = ("SELECT * FROM PERSONAL_EXPENSES WHERE Cashback > 500")

my_cursor.execute(query12)

transaction_with_cashback = my_cursor.fetchall()

for x in transaction_with_cashback:
    
    print(x)
    
columns = [desc[0] for desc in my_cursor.description]


cashback_df =pd.DataFrame(transaction_with_cashback, columns=columns)

fig = px.bar(cashback_df,
             x = "Description",
             y = "Cashback",
             title="Transaction with Cashback Greater Than 500",
             color="Cashback",
             text="Cashback",
             width=1000,
             height=800
)

fig.show()

(12, datetime.date(2023, 1, 6), 'Housing', 'Cash', 'Furnishings', Decimal('18344.52'), Decimal('671.39'))
(22, datetime.date(2023, 1, 8), 'Housing', 'Cash', 'Rent Payment', Decimal('29967.21'), Decimal('2478.23'))
(39, datetime.date(2023, 1, 13), 'Housing', 'Online', 'Appliance Purchases', Decimal('8795.42'), Decimal('711.57'))
(73, datetime.date(2023, 1, 22), 'Housing', 'Cash', 'Furnishings', Decimal('18961.09'), Decimal('1802.73'))
(101, datetime.date(2023, 2, 1), 'Housing', 'Online', 'Furnishings', Decimal('12188.60'), Decimal('637.22'))
(130, datetime.date(2023, 2, 9), 'Housing', 'Online', 'Appliance Purchases', Decimal('25641.91'), Decimal('1689.63'))
(145, datetime.date(2023, 2, 14), 'Travel', 'Online', 'Hotel Accommodation', Decimal('16824.46'), Decimal('1205.01'))
(151, datetime.date(2023, 2, 15), 'Housing', 'Cash', 'Furnishings', Decimal('17446.31'), Decimal('1481.43'))
(170, datetime.date(2023, 2, 19), 'Housing', 'Cash', 'Mortgage Payment', Decimal('14438.82'), Decimal('1370.

In [23]:
# Daily cashback trend

query13 = ("SELECT Date, SUM(Cashback) FROM PERSONAL_EXPENSES GROUP BY Date ORDER BY Date")

my_cursor.execute(query13)

Daily_Cashback_Trend = my_cursor.fetchall()

for x in Daily_Cashback_Trend:
    print(x)

cashback_trend_df = pd.DataFrame(Daily_Cashback_Trend, columns=['Date','Total Cashback'])

fig = px.line(
    cashback_trend_df,
    x='Date',
    y='Total Cashback',
    title='Daily Cashback Trend',
    markers=True,
    color_discrete_sequence=px.colors.sequential.Plasma
    )

fig.update_traces(
    line=dict(color="#2d3d5c", width=3),
    marker=dict(color="#ffccd5", size=5))

fig.show()

(datetime.date(2023, 1, 1), Decimal('502.78'))
(datetime.date(2023, 1, 2), Decimal('67.60'))
(datetime.date(2023, 1, 3), Decimal('230.82'))
(datetime.date(2023, 1, 4), Decimal('28.82'))
(datetime.date(2023, 1, 5), Decimal('162.16'))
(datetime.date(2023, 1, 6), Decimal('1506.94'))
(datetime.date(2023, 1, 7), Decimal('81.66'))
(datetime.date(2023, 1, 8), Decimal('5632.72'))
(datetime.date(2023, 1, 9), Decimal('450.36'))
(datetime.date(2023, 1, 10), Decimal('939.14'))
(datetime.date(2023, 1, 11), Decimal('13.28'))
(datetime.date(2023, 1, 12), Decimal('305.26'))
(datetime.date(2023, 1, 13), Decimal('2322.20'))
(datetime.date(2023, 1, 14), Decimal('117.54'))
(datetime.date(2023, 1, 15), Decimal('565.54'))
(datetime.date(2023, 1, 16), Decimal('937.18'))
(datetime.date(2023, 1, 17), Decimal('814.22'))
(datetime.date(2023, 1, 18), Decimal('1033.42'))
(datetime.date(2023, 1, 19), Decimal('495.82'))
(datetime.date(2023, 1, 20), Decimal('350.54'))
(datetime.date(2023, 1, 21), Decimal('482.38'))
(